<a href="https://colab.research.google.com/github/cristinatruta/spam-messages/blob/main/Proiect_bun_spamipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Information
The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...

I used the dataset from kaggle.com, spam.csv It has 5169 unique values.

Import modules

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

Loading the dataset

In [ ]:
df = pd.read_csv(r"/content/sample_data/spam.csv",)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
# get necessary columns for processing
df = df[['v2', 'v1']]
# df.rename(columns={'v2': 'messages', 'v1': 'label'}, inplace=True)
df = df.rename(columns={'v2': 'messages', 'v1': 'label'})
df.head()

,messages,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
df['Spam']=df['label'].apply(lambda x:1 if x=='spam' else 0)
df.head(5)

,messages,label,Spam
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


Preprocessing the dataset

In [ ]:
# check for null values
df.isnull().sum()

messages    0
label       0
Spam        0
dtype: int64

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
STOPWORDS =set(stopwords.words('english'))
def clean_text(text):
    # convert to lowercase
    text = text.lower()
    # remove special characters
    text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # remove stopwords
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [ ]:
# clean the messages
df['clean_text'] = df['messages'].apply(clean_text)
df.head()

,messages,label,Spam,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,0,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ham,0,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,ham,0,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,0,nah think goes usf lives around though


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.messages,df.Spam,test_size=0.25)

In [ ]:
#CounterVectorizer Convert the text into matrics
from sklearn.feature_extraction.text import CountVectorizer

Naive Bayes has three Classifiers (Bernoulli, Multinomial, Gaussian) We use Multinomial Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# model training
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

Training The Model

In [ ]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [73]:
#classification report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1219
           1       0.97      0.87      0.92       174

    accuracy                           0.98      1393
   macro avg       0.98      0.93      0.95      1393
weighted avg       0.98      0.98      0.98      1393



In [69]:
emails=[
    'Hello, are you home, today?',
    'Will u meet ur dream partner soon? Is ur career off 2 a flyng start? 2 find out free, txt HORO followed by ur star sign, e. g. HORO ARIES'
]

In [70]:
clf.predict(emails)

array([0, 1])

Prediction Of Model

In [71]:
print("Accuracy:" , clf.score(X_test,y_test)*100)

Accuracy: 98.06173725771716
